# Context Construction Best Practices

This notebook demonstrates four key principles for effective prompt engineering:

1. **Position matters**: Models process information at the beginning and end of prompts better than content in the middle
2. **Be mindful of context limits**: Track token usage and prioritize the most relevant information
3. **Dynamic context**: Tailor context to each query rather than using static templates
4. **Format appropriately**: Present data as concise summaries rather than raw dumps

Let's explore each principle with practical examples!

In [ ]:
# Setup and imports
!pip install litellm tiktoken
from litellm import completion
import tiktoken
import json
from datetime import datetime

In [ ]:
# Model configuration
model = "ollama/driaforall/tiny-agent-a:0.5b"

# Helper function to count tokens (approximate)
def count_tokens(text):
    """Approximate token count"""
    try:
        encoding = tiktoken.get_encoding("cl100k_base")
        return len(encoding.encode(str(text)))
    except:
        # Fallback approximation
        return len(str(text)) // 4

def get_response(messages, show_tokens=True):
    """Helper function to get response and show token usage"""
    if show_tokens:
        total_tokens = sum(count_tokens(msg["content"]) for msg in messages)
        print(f"Estimated tokens used: {total_tokens}")
    
    response = completion(model=model, messages=messages)
    return response.choices[0].message.content

## 1. Position Matters: Beginning and End Are Key

Models exhibit **recency bias** and **primacy bias** - they pay more attention to information at the beginning and end of the context than content buried in the middle.

###  Bad Example: Important Information Buried

In [ ]:
# Bad: Critical information hidden in the middle
bad_messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": """
    I need help with my project. Here's some background: I work at a tech company.
    We use various tools and technologies. Our team is diverse and talented.
    The project involves machine learning and data processing.
    CRITICAL: The deadline is tomorrow and I need to focus only on the deployment checklist.
    We've been working on this for months. The stakeholders are eager to see results.
    What should I do to prepare for the presentation?
    """}
]

print(" Bad Example Response:")
bad_response = get_response(bad_messages)
print(bad_response)
print("\n" + "="*50 + "\n")

###  Good Example: Critical Information at Beginning and End

In [ ]:
# Good: Critical information prominently positioned
good_messages = [
    {"role": "system", "content": """
    You are a deployment specialist. PRIORITY: Focus on actionable deployment checklists.
    """}, 
    {"role": "user", "content": """
    URGENT: Deployment deadline is tomorrow. Need deployment checklist focus only.
    
    Context: ML project at tech company, months of development, stakeholder presentation.
    
    QUESTION: What deployment checklist items should I prioritize for tomorrow's deadline?
    """}
]

print("Good Example Response:")
good_response = get_response(good_messages)
print(good_response)

## 2. Context Limits: Token Management

Most models have context windows (e.g., 4K, 8K, 32K tokens). Exceeding these limits causes truncation or errors.

### Demonstration: Tracking Token Usage

In [ ]:
# Example: Building context while monitoring tokens
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

print("Token usage as we build context:")
print(f"System prompt: {count_tokens(messages[0]['content'])} tokens")

# Add user message
user_msg = "What are the best practices for API design?"
messages.append({"role": "user", "content": user_msg})
print(f"+ User message: {count_tokens(user_msg)} tokens")
print(f"Total so far: {sum(count_tokens(msg['content']) for msg in messages)} tokens")

# Simulate adding large context (like documentation)
large_context = "Here's our entire API documentation: " + "API endpoint details. " * 200
print(f"\nLarge context would add: {count_tokens(large_context)} tokens")
print(" This might exceed context limits!")

# Better approach: Summarized context
summarized_context = """
Key API principles from docs:
- RESTful design patterns
- Consistent error handling
- Rate limiting implementation
- Authentication best practices
"""
print(f"\nSummarized context: {count_tokens(summarized_context)} tokens")
print(" Much more efficient!")

## 2.1. Managing Conversation History in Context

Including previous messages maintains conversation continuity, but requires careful management to avoid exceeding context limits.

###  Bad Example: Including All History Without Management

In [ ]:
# Bad: Blindly including all conversation history
def bad_conversation_management():
    # Simulated long conversation history
    conversation_history = [
        {"role": "system", "content": "You are a helpful coding assistant."},
        {"role": "user", "content": "What's the difference between lists and tuples in Python?"},
        {"role": "assistant", "content": "Lists are mutable (can be changed) while tuples are immutable (cannot be changed). Lists use square brackets [] and tuples use parentheses (). Lists are better for data that might change, tuples for fixed data."},
        {"role": "user", "content": "Can you show me some examples of when to use each?"},
        {"role": "assistant", "content": "Sure! Use lists for: shopping lists, user data that updates, dynamic collections. Use tuples for: coordinates (x,y), database records, configuration settings, function return values when returning multiple items."},
        {"role": "user", "content": "What about performance differences?"},
        {"role": "assistant", "content": "Tuples are generally faster for accessing elements and take less memory. Lists are slower due to their mutability features but more flexible. For large datasets where you don't need to modify data, tuples can be significantly more efficient."},
        {"role": "user", "content": "How do I convert between them?"},
        {"role": "assistant", "content": "Use list() to convert tuple to list: list((1,2,3)) gives [1,2,3]. Use tuple() to convert list to tuple: tuple([1,2,3]) gives (1,2,3). Both create new objects, so original is unchanged."},
        # ... many more messages
    ]
    
    # New question - including ALL history
    new_question = "Now I want to learn about dictionaries in Python."
    
    # This creates a very long context
    full_messages = conversation_history + [{"role": "user", "content": new_question}]
    
    total_tokens = sum(count_tokens(msg['content']) for msg in full_messages)
    print(f" Bad approach - Total tokens: {total_tokens}")
    print(f"Number of messages: {len(full_messages)}")
    print(" This will quickly exceed context limits!")
    
    return full_messages

bad_messages = bad_conversation_management()

###  Good Example: Smart Conversation History Management

In [ ]:
# Good: Intelligent conversation history management
def smart_conversation_management(conversation_history, new_question, max_history_tokens=500):
    """
    Manage conversation history intelligently:
    1. Keep system prompt
    2. Summarize or truncate old messages
    3. Keep recent context
    4. Stay within token limits
    """
    
    # Always keep the system prompt
    system_prompt = conversation_history[0]
    recent_messages = conversation_history[1:]  # All messages except system
    
    # Strategy 1: Keep only recent messages that fit in token limit
    selected_messages = [system_prompt]
    current_tokens = count_tokens(system_prompt['content'])
    
    # Add messages from most recent backwards until we hit limit
    for message in reversed(recent_messages):
        msg_tokens = count_tokens(message['content'])
        if current_tokens + msg_tokens <= max_history_tokens:
            selected_messages.insert(1, message)  # Insert after system prompt
            current_tokens += msg_tokens
        else:
            break
    
    # Add new question
    selected_messages.append({"role": "user", "content": new_question})
    
    return selected_messages, current_tokens

# Test with the same conversation
conversation_history = [
    {"role": "system", "content": "You are a helpful coding assistant."},
    {"role": "user", "content": "What's the difference between lists and tuples in Python?"},
    {"role": "assistant", "content": "Lists are mutable, tuples are immutable. Lists use [], tuples use ()."},
    {"role": "user", "content": "Can you show me examples?"},
    {"role": "assistant", "content": "Lists: shopping_list = ['milk', 'eggs']. Tuples: coordinates = (10, 20)."},
    {"role": "user", "content": "What about performance differences?"},
    {"role": "assistant", "content": "Tuples are faster and use less memory due to immutability."},
]

new_question = "Now I want to learn about dictionaries in Python."

smart_messages, tokens_used = smart_conversation_management(conversation_history, new_question)

print(" Smart conversation management:")
print(f"Selected messages: {len(smart_messages)}")
print(f"Total tokens: {tokens_used + count_tokens(new_question)}")
print("\nMessage structure:")
for i, msg in enumerate(smart_messages):
    role = msg['role']
    content_preview = msg['content'][:50] + "..." if len(msg['content']) > 50 else msg['content']
    print(f"{i+1}. {role.title()}: {content_preview}")

### Advanced Strategy: Conversation Summarization

In [ ]:
# Advanced: Summarize older conversation parts
def advanced_conversation_management(conversation_history, new_question, max_tokens=800):
    """
    Advanced strategy:
    1. Keep system prompt
    2. Summarize older conversation parts
    3. Keep recent detailed exchanges
    4. Add new question
    """
    
    system_prompt = conversation_history[0]
    messages = conversation_history[1:]
    
    # Keep last 4 messages (2 exchanges) for full context
    recent_messages = messages[-4:] if len(messages) >= 4 else messages
    older_messages = messages[:-4] if len(messages) >= 4 else []
    
    # Create summary of older conversation
    if older_messages:
        # Extract key topics from older messages
        topics = []
        for msg in older_messages:
            if msg['role'] == 'user':
                # Extract main topic from user questions
                content = msg['content'].lower()
                if 'list' in content and 'tuple' in content:
                    topics.append("lists vs tuples")
                elif 'example' in content:
                    topics.append("practical examples")
        
        summary = f"Previous discussion covered: {', '.join(set(topics))}"
        summary_message = {"role": "system", "content": f"Context: {summary}"}
    else:
        summary_message = None
    
    # Build final message list
    final_messages = [system_prompt]
    
    if summary_message:
        final_messages.append(summary_message)
    
    final_messages.extend(recent_messages)
    final_messages.append({"role": "user", "content": new_question})
    
    total_tokens = sum(count_tokens(msg['content']) for msg in final_messages)
    
    return final_messages, total_tokens

# Test advanced strategy
advanced_messages, advanced_tokens = advanced_conversation_management(conversation_history, new_question)

print(" Advanced conversation management:")
print(f"Total messages: {len(advanced_messages)}")
print(f"Total tokens: {advanced_tokens}")
print("\nStructure:")
for i, msg in enumerate(advanced_messages):
    role = msg['role']
    content_preview = msg['content'][:60] + "..." if len(msg['content']) > 60 else msg['content']
    tokens = count_tokens(msg['content'])
    print(f"{i+1}. {role.title()} ({tokens} tokens): {content_preview}")

# Get response using advanced strategy
print("\n Response with proper conversation context:")
advanced_response = get_response(advanced_messages, show_tokens=False)
print(advanced_response[:200] + "..." if len(advanced_response) > 200 else advanced_response)

### Key Strategies for Conversation History:

1. **Always preserve the system prompt** - It defines the assistant's behavior

2. **Prioritize recent messages** - They provide immediate context

3. **Summarize older exchanges** - Capture key topics without full detail

4. **Monitor token usage** - Set limits and stick to them

5. **Consider message relevance** - Some older messages may be more important than recent ones

### Token Comparison:
-  **All history**: Can easily exceed 2000+ tokens
-  **Smart selection**: ~300-500 tokens
-  **Advanced summarization**: ~400-600 tokens with better context


## 3. Dynamic vs Static Context

Tailor your context to each specific query rather than using one-size-fits-all templates.

###  Static Context Example

In [ ]:
# Static context - same for all queries
static_system_prompt = """
You are a coding assistant. You help with Python, JavaScript, API design, 
database queries, debugging, testing, deployment, security, performance, 
documentation, code review, and general software development questions.
"""

# Query 1: Simple Python question
query1_messages = [
    {"role": "system", "content": static_system_prompt},
    {"role": "user", "content": "How do I reverse a string in Python?"}
]

print(" Static context for simple Python question:")
print(f"System prompt tokens: {count_tokens(static_system_prompt)}")
response1 = get_response(query1_messages, show_tokens=False)
print(f"Response: {response1[:100]}...")
print("\n" + "-"*30 + "\n")

###  Dynamic Context Example

In [ ]:
# Dynamic context - tailored to specific query
def create_dynamic_context(query_type, specific_context=""):
    base_contexts = {
        "python_basics": "You are a Python tutor. Focus on clear, beginner-friendly explanations with examples.",
        "api_design": "You are an API architect. Focus on RESTful principles, best practices, and scalability.",
        "debugging": "You are a debugging expert. Focus on systematic troubleshooting approaches.",
        "security": "You are a security specialist. Focus on secure coding practices and threat mitigation."
    }
    
    context = base_contexts.get(query_type, "You are a helpful assistant.")
    if specific_context:
        context += f" Additional context: {specific_context}"
    
    return context

# Same query with dynamic context
dynamic_system_prompt = create_dynamic_context("python_basics")

query1_dynamic = [
    {"role": "system", "content": dynamic_system_prompt},
    {"role": "user", "content": "How do I reverse a string in Python?"}
]

print(" Dynamic context for simple Python question:")
print(f"System prompt tokens: {count_tokens(dynamic_system_prompt)}")
response1_dynamic = get_response(query1_dynamic, show_tokens=False)
print(f"Response: {response1_dynamic[:100]}...")

print(f"\n Token savings: {count_tokens(static_system_prompt) - count_tokens(dynamic_system_prompt)} tokens")

## 4. Proper Formatting: Summaries vs Raw Dumps

Present information in digestible, structured formats rather than overwhelming raw data.

###  Bad Example: Raw Data Dump

In [ ]:
# Simulated raw database results
raw_data = """
Database results:
id: 1, name: John Smith, email: john@example.com, department: Engineering, salary: 85000, hire_date: 2022-01-15, manager_id: 5, project_count: 3, last_review: 2023-08-10, performance_rating: 4.2
id: 2, name: Sarah Johnson, email: sarah@example.com, department: Engineering, salary: 92000, hire_date: 2021-03-22, manager_id: 5, project_count: 5, last_review: 2023-07-25, performance_rating: 4.7
id: 3, name: Mike Chen, email: mike@example.com, department: Marketing, salary: 78000, hire_date: 2023-02-01, manager_id: 8, project_count: 2, last_review: 2023-09-15, performance_rating: 3.9
"""

bad_context_messages = [
    {"role": "system", "content": "You are an HR assistant."},
    {"role": "user", "content": f"{raw_data}\n\nWho are our top performers in Engineering?"}
]

print(" Bad: Raw data dump")
print(f"Context size: {count_tokens(raw_data)} tokens")
bad_formatting_response = get_response(bad_context_messages, show_tokens=False)
print(f"Response: {bad_formatting_response}")
print("\n" + "-"*50 + "\n")

## Good Example

In [ ]:
# Well-formatted summary
structured_summary = """
Engineering Team Summary:
- John Smith: 4.2 rating, 3 projects, hired Jan 2022
- Sarah Johnson: 4.7 rating, 5 projects, hired Mar 2021

Performance Metrics:
- Average rating: 4.45
- Total projects: 8
- Team size: 2
"""

good_context_messages = [
    {"role": "system", "content": "You are an HR assistant specializing in performance analysis."},
    {"role": "user", "content": f"{structured_summary}\n\nWho are our top performers in Engineering?"}
]

print(" Good: Structured summary")
print(f"Context size: {count_tokens(structured_summary)} tokens")
good_formatting_response = get_response(good_context_messages, show_tokens=False)
print(f"Response: {good_formatting_response}")

print(f"\n Token savings: {count_tokens(raw_data) - count_tokens(structured_summary)} tokens")
print(f" Efficiency gain: {((count_tokens(raw_data) - count_tokens(structured_summary)) / count_tokens(raw_data) * 100):.1f}%")

## 5. Context Components in Action

Let's see how different context components work together effectively.

### Complete Context Example

In [ ]:
# Simulating a complete context with all components
def build_complete_context(user_query, conversation_history=None, retrieved_info=None, available_tools=None):
    
    # 1. System prompt (defines role and behavior)
    system_prompt = """
    You are a senior software architect. You provide technical guidance based on best practices,
    consider scalability, maintainability, and security in your recommendations.
    """
    
    messages = [{"role": "system", "content": system_prompt.strip()}]
    
    # 2. Add conversation history (if any)
    if conversation_history:
        messages.extend(conversation_history)
    
    # 3. Build user message with retrieved info and tools
    user_content = ""
    
    # Add retrieved information (RAG)
    if retrieved_info:
        user_content += f"Relevant context:\n{retrieved_info}\n\n"
    
    # Add available tools
    if available_tools:
        user_content += f"Available tools: {', '.join(available_tools)}\n\n"
    
    # Add the actual user query
    user_content += f"Question: {user_query}"
    
    messages.append({"role": "user", "content": user_content})
    
    return messages

# Example usage
conversation_history = [
    {"role": "user", "content": "I'm building a social media app."},
    {"role": "assistant", "content": "Great! What's your main technical challenge?"}
]

retrieved_info = """
Database performance tips:
- Use connection pooling
- Implement read replicas
- Add proper indexing
"""

available_tools = ["database_optimizer", "load_tester", "security_scanner"]

complete_messages = build_complete_context(
    user_query="How should I optimize my database for handling user feeds?",
    conversation_history=conversation_history,
    retrieved_info=retrieved_info,
    available_tools=available_tools
)

print(" Complete Context Structure:")
for i, msg in enumerate(complete_messages):
    print(f"{i+1}. {msg['role'].title()}: {count_tokens(msg['content'])} tokens")
    if msg['role'] == 'user':
        print(f"   Preview: {msg['content'][:100]}...")

print("\n Response:")
complete_response = get_response(complete_messages)
print(complete_response)

## 6. Practical Exercises

Now it's your turn to practice! Complete these exercises to reinforce your learning.

### Exercise 1: Fix the Position Problem

In [ ]:
# Exercise 1: Rewrite this poorly positioned context
poorly_positioned = [
    {"role": "system", "content": "You are an assistant."},
    {"role": "user", "content": """
    I'm working on various projects. Some involve web development, others mobile apps.
    My team uses different technologies and frameworks. We have deadlines to meet.
    URGENT: I need to fix a critical security vulnerability in our authentication system NOW.
    The system handles user login and session management. We use JWT tokens.
    What general development tips do you have?
    """}
]

# TODO: Rewrite to properly position the critical information
# Your solution here:
fixed_positioning = [
    {"role": "system", "content": "You are a security specialist. PRIORITY: Address critical vulnerabilities immediately."},
    {"role": "user", "content": """
    CRITICAL SECURITY ISSUE: Authentication vulnerability needs immediate fix.
    
    System details: JWT-based auth, handles login/sessions
    Context: Web/mobile projects, team environment
    
    URGENT QUESTION: How do I fix this authentication vulnerability?
    """}
]

print("Exercise 1 - Your Solution:")
exercise1_response = get_response(fixed_positioning)
print(exercise1_response)

### Exercise 2: Create Dynamic Context

In [ ]:
# Exercise 2: Create a dynamic context system
def create_context_for_query(query, domain):
    """
    Create appropriate context based on the query and domain.
    
    Args:
        query: The user's question
        domain: The domain (e.g., 'security', 'performance', 'ui_ux', 'testing')
    
    Returns:
        List of messages with appropriate context
    """
    # TODO: Implement dynamic context creation
    context_templates = {
        'security': 'You are a cybersecurity expert. Focus on threat analysis and secure coding practices.',
        'performance': 'You are a performance optimization specialist. Focus on efficiency and scalability.',
        'ui_ux': 'You are a UX designer. Focus on user experience and interface design principles.',
        'testing': 'You are a QA engineer. Focus on testing strategies and quality assurance.'
    }
    
    system_content = context_templates.get(domain, 'You are a helpful assistant.')
    
    return [
        {"role": "system", "content": system_content},
        {"role": "user", "content": query}
    ]

# Test your function
test_queries = [
    ("How do I prevent SQL injection?", "security"),
    ("My app is loading slowly, what should I check?", "performance"),
    ("How can I make my form more user-friendly?", "ui_ux")
]

print("Exercise 2 - Testing Dynamic Context:")
for query, domain in test_queries:
    messages = create_context_for_query(query, domain)
    print(f"\nDomain: {domain}")
    print(f"Tokens: {sum(count_tokens(msg['content']) for msg in messages)}")
    response = get_response(messages, show_tokens=False)
    print(f"Response preview: {response[:100]}...")